In [1]:
%matplotlib inline
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
from pandas.plotting import lag_plot
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from lib.metamodels import Metamodels
from scipy import stats, integrate
import seaborn as sns

sns.set(color_codes=True)
plt.rcParams['figure.figsize'] = [15, 10]

/Users/nlong/.pyenv/versions/3.7.7-metamodel/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


ModuleNotFoundError: No module named 'lib.metamodels'

In [ ]:
analysis_id = '66fb9766-26e7-4bed-bdf9-0fbfbc8d6c7e'
data_file = 'output/%s/simulation_results.csv' % analysis_id
if not os.path.exists(data_file):
    raise Exception("path does not exist")

print("Loaded data file: %s" % data_file)

 # data_file_to_csv()
dataset = pd.read_csv(data_file)
# this column is a redundant column
dataset = dataset.drop('DistrictCoolingOutletTemperature', 1)
# update some of the column names so they make sense to this model
dataset = dataset.rename(columns={
    'DistrictHeatingOutletTemperature': 'ETSInletTemperature',
    'DistrictHeatingInletTemperature': 'ETSHeatingOutletTemperature',
    'DistrictCoolingInletTemperature': 'ETSCoolingOutletTemperature',
})
# This command takes awhile to run... hang tight.
dataset['DateTime'] = pd.to_datetime(dataset['DateTime'])

In [ ]:
dataset.columns.values

In [ ]:
# load in the metamodels
rom = Metamodels('./metamodels.json')
rom.set_analysis(analysis_id)
# Load the exising models - If passed with list, then it will only load the models in the list.
rom.load_models()


In [ ]:
dataset._id.describe()

In [ ]:
temperature_vars = [
       'ETSInletTemperature',
       'SiteOutdoorAirDrybulbTemperature',
       'ETSHeatingOutletTemperature',
       'ETSCoolingOutletTemperature',
       'ambient_loop_temperature_setpoint.setpoint_temperature',
       'ambient_loop_temperature_setpoint.design_delta',
     ]
temperature_series = dataset[temperature_vars]
temperature_series.describe()



In [ ]:
temperature_series.plot.box()
plt.ylabel("Temperature (deg C)")
plt.xticks(rotation=10)

In [ ]:
mass_flow_series = dataset[['DistrictCoolingMassFlowRate']]
print mass_flow_series[mass_flow_series.DistrictCoolingMassFlowRate > 0].describe()
mass_flow_series[mass_flow_series.DistrictCoolingMassFlowRate > 0].plot.box()
plt.ylabel("Cooling Mass Flow Rate (kg/s)")


In [ ]:
mass_flow_series = dataset[['DistrictHeatingMassFlowRate']]
print mass_flow_series[mass_flow_series.DistrictHeatingMassFlowRate > 0].describe()
mass_flow_series[mass_flow_series.DistrictHeatingMassFlowRate > 0].plot.box()
plt.ylabel("Heating Mass Flow Rate (kg/s)")

In [ ]:
# get the time series data for 3/15
dataset.set_index('DateTime')
dataset.DateTime.describe()
# dataset.DateTime[15]

In [ ]:
ds = dataset[dataset["DateTime"].between("2009-03-01 01:00", "2009-03-10 00:00")]

# constrain to a specific building
ds_single_building = ds[ds._id == '606171e5-3517-4ab1-85fe-276c34014eae']

# Add in the ROM results into the single building
# ds_single_building['RF_HeatingElectricity'] = rom.yhat('HeatingElectricity', ds_single_building)
ds_single_building['RF_ETSHeatingOutletTemperature'] = rom.yhat('ETSHeatingOutletTemperature', ds_single_building)
ds_single_building.iloc[0]

In [ ]:
sns.set(style="darkgrid")
temps_single_buildings = ds_single_building[
    temperature_vars + ['DateTime', 'RF_ETSHeatingOutletTemperature', 'DistrictHeatingMassFlowRate', 'DistrictCoolingMassFlowRate']
]
temps_single_buildings.describe()

In [ ]:
# plot the RF ETS Outlet  with the ETSOutput
newplt = sns.lmplot(x="ETSHeatingOutletTemperature", y="RF_ETSHeatingOutletTemperature", data=temps_single_buildings,
           ci=None, palette="muted", height=8, scatter_kws={"s": 50, "alpha": 1})
newplt = newplt.axes.flatten()
newplt[0].set_title("RF vs EnergyPlus Outlet Temperature")

In [ ]:
#selected_columns = ['DateTime', 'ETSHeatingOutletTemperature','RF_ETSHeatingOutletTemperature', 'DistrictHeatingMassFlowRate', 'DistrictCoolingMassFlowRate']
selected_columns = ['DateTime', 'ETSHeatingOutletTemperature','RF_ETSHeatingOutletTemperature']
melted_df = pd.melt(temps_single_buildings[selected_columns], id_vars='DateTime', var_name='Variable', value_name='Value')
melted_df['Dummy'] = 0

def date_formatter(x, pos):
    return pd.to_datetime(x)

fig, ax = plt.subplots()

newplt = sns.tsplot(melted_df, time='DateTime', unit='Dummy', condition='Variable', value='Value', ax=ax)
newplt.set_title("RF vs EnergyPlus Heating Outlet Temperature")
ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(date_formatter))

# put the labels at 45deg since they tend to be too long
fig.autofmt_xdate()
plt.show()

In [ ]:
ds_single_building.columns.values



In [ ]:
d = pd.DataFrame(data=ds_single_building,
                 columns=[
                     'SiteOutdoorAirDrybulbTemperature',
                     'ETSInletTemperature',
                     'ETSHeatingOutletTemperature',
                     'DistrictHeatingHotWaterEnergy',
                     'DistrictHeatingMassFlowRate',
                     'DistrictCoolingMassFlowRate',
                     'HeatingElectricity',
                 ])

# Compute the correlation matrix
corr = d.corr()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
# TODO: move this over to the python source code
filename = 'output/66fb9766-26e7-4bed-bdf9-0fbfbc8d6c7e/lookup_tables/heating_ETSHeatingOutletTemperature_mass_flow_0.05.csv'
df_heatmap = pd.read_csv(filename, header=0)
# Remove the datetime column before converting the column headers to rounded floats
df_heatmap = df_heatmap.drop(columns=['datetime'])
df_heatmap.rename(columns=lambda x: round(float(x), 1), inplace=True)
# print df_heatmap

# try creating heatmap
import matplotlib.pyplot as plt
import seaborn as sns

# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(5, 12))

sns.heatmap(df_heatmap)
ax.set_title('ETS Outlet Temperature - mf 0.05 kg/s')
ax.set_xlabel('ETS Inlet Temperature')
ax.set_ylabel('Hour of Year')
